Ref: https://github.com/NCAR/cesm-lens-aws/issues/34

In [14]:
import xarray as xr
import intake
import intake_esm
import zarr
import dask
from ncar_jobqueue import NCARCluster
from distributed import Client
from distributed.utils import format_bytes

xr.set_options(display_style='html')
#dask.config.set({'distributed.dashboard.link': '/proxy/{port}/status'})

In [2]:
dask.config.get('distributed.dashboard')

{'link': 'https://jupyterhub.ucar.edu/dav/user/{USER}/proxy/{port}/status',
 'export-tool': False}

In [3]:
# cluster = NCARCluster()
# cluster.adapt(minimum_jobs=2)
# client = Client(cluster)
# cluster

In [4]:
col = intake.open_esm_datastore("./GitRepos/intake-esm-datastore/catalogs/glade-cesm1-le.json")
col

glade-cesm1-le-ESM Collection with 191066 entries:
	> 7 experiment(s)

	> 108 case(s)

	> 6 component(s)

	> 15 stream(s)

	> 1052 variable(s)

	> 116 date_range(s)

	> 40 member_id(s)

	> 191066 path(s)

	> 6 ctrl_branch_year(s)

	> 4 ctrl_experiment(s)

	> 41 ctrl_member_id(s)

In [5]:
variables = [
    "THETA",
    "UVEL",
    "UVEL2",
    "VVEL",
    "VVEL2",
    "WVEL",
    "FW",
    "HDIFB_SALT",
    "HDIFB_TEMP",
    "HDIFE_SALT",
    "HDIFE_TEMP",
    "HDIFN_SALT",
    "HDIFN_TEMP",
    "HMXL",
    "HOR_DIFF",
    "KAPPA_ISOP",
    "KAPPA_THIC",
    "KPP_SRC_SALT",
    "KPP_SRC_TEMP",
    "RESID_S",
    "RESID_T",
    "QFLUX",
    "SHF",
    "SHF_QSW",
    "SFWF",
    "SFWF_WRST",
    "SSH",
    "TAUX",
    "TAUX2",
    "TAUY",
    "TAUY2",
    "VNT_ISOP",
    "VNT_SUBM",
    "UES",
    "UET",
    "VNS",
    "VNT",
    "WTS",
    "WTT",
]

In [6]:
col.search(component='ocn').unique(columns=['stream', 'experiment'])

{'stream': {'count': 4,
  'values': ['pop.h.ecosys.nyear1',
   'pop.h.ecosys.nday1',
   'pop.h.nday1',
   'pop.h']},
 'experiment': {'count': 5,
  'values': ['CTRL', '20C', '20C-OIC', 'RCP85', 'RCP45']}}

In [22]:
col_subset = col.search(component='ocn', variable=variables[:6], 
                        experiment=['20C', 'RCP85', 'CTRL'])
col_subset

glade-cesm1-le-ESM Collection with 655 entries:
	> 3 experiment(s)

	> 81 case(s)

	> 1 component(s)

	> 1 stream(s)

	> 5 variable(s)

	> 23 date_range(s)

	> 40 member_id(s)

	> 655 path(s)

	> 4 ctrl_branch_year(s)

	> 3 ctrl_experiment(s)

	> 41 ctrl_member_id(s)

In [23]:
col_subset.unique(columns=['stream', 'variable'])

{'stream': {'count': 1, 'values': ['pop.h']},
 'variable': {'count': 5,
  'values': ['UVEL', 'UVEL2', 'VVEL', 'VVEL2', 'WVEL']}}

In [24]:
dsets = col_subset.to_dataset_dict(cdf_kwargs={'chunks': {'time': 12, 'z_t': 30}})

datasets = {}
for key, dset in dsets.items():
    dset = dset.copy()
    for v in dset.variables:
        if v not in variables and v not in dset.coords:
            dset[v] = dset[v].isel(time=0).drop('time')
    datasets[key] = dset

Progress: |███████████████████████████████████████████████████████████████████████████████| 100.0% 

--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.variable'
             
--> There are 15 group(s)


In [34]:
total_size = 0
for key, ds in datasets.items():
    print(f'key={key} size={format_bytes(ds.nbytes)}')
    total_size = total_size + ds.nbytes
    
print(f'Total size = {format_bytes(total_size)}')

key=ocn.CTRL.pop.h.VVEL2 size=637.38 GB
key=ocn.CTRL.pop.h.WVEL size=637.38 GB
key=ocn.CTRL.pop.h.UVEL2 size=637.38 GB
key=ocn.CTRL.pop.h.UVEL size=637.38 GB
key=ocn.CTRL.pop.h.VVEL size=637.38 GB
key=ocn.20C.pop.h.UVEL2 size=2.21 TB
key=ocn.20C.pop.h.WVEL size=2.21 TB
key=ocn.20C.pop.h.VVEL size=2.21 TB
key=ocn.20C.pop.h.UVEL size=2.21 TB
key=ocn.20C.pop.h.VVEL2 size=2.21 TB
key=ocn.RCP85.pop.h.VVEL size=1.34 TB
key=ocn.RCP85.pop.h.UVEL2 size=1.34 TB
key=ocn.RCP85.pop.h.WVEL size=1.34 TB
key=ocn.RCP85.pop.h.VVEL2 size=1.34 TB
key=ocn.RCP85.pop.h.UVEL size=1.34 TB
Total size = 20.95 TB
